In [22]:
import pandas as pd, numpy as np
import random
from pulp import *
from ortoolpy import addbinvars

In [23]:
#市橋先生：5/3日直OC

# Collect response
# Parameters
address_response = "https://docs.google.com/spreadsheets/d/1RdQY60-y0Ml4-Hi2AUoLGvXqQZpAHzu3XeYTdYDvJ_g/edit?usp=sharing"
name_sheet = "FormResponses1"
l_member_main = ['藤川 慎也','池亀 天平','越山 太輔','熊倉 陽介','森田 進', '水谷 真志', '清田 正紘','田中 李樹']
l_member_sub = ['田尻 智哉','佐藤 未悠','南學 正仁','相澤 里佳','押見 陽友']
l_duty = ['5/2(金)当直', '5/3(土)日直', '5/3(土)当直', '5/4(日)日直', '5/4(日)当直', '5/5(月)日直', '5/5(月)当直', '5/6(火)日直', '5/6(火)当直']
l_rank = ['第1希望', '第2希望', '第3希望', '第4希望', '第5希望', '第6希望', '第7希望', '第8希望', '第9希望']

# Read G sheet
sheet_id = address_response.split('/')[5]
d_preference_src = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={name_sheet}")

# Convert
def convert_preference(d_preference_src, l_member, l_duty, l_rank):    
    d_preference = pd.DataFrame(index = [i for i in range(len(l_duty))], columns = [i for i in range(len(l_member))])
    for i_member, name_member in enumerate(l_member):
        if name_member in d_preference_src['お名前（敬称略）'].tolist():
            s_preference_member = d_preference_src.loc[d_preference_src['お名前（敬称略）'] == name_member, [col.startswith('日当直希望') for col in d_preference_src.columns]]
            s_preference_member = s_preference_member.iloc[len(s_preference_member)-1, :]
            for i_duty, name_duty in enumerate(l_duty):
                rank_src = s_preference_member[[name_duty in i_pref for i_pref in s_preference_member.index.tolist()]].tolist()[0]
                for i_rank, name_rank in enumerate(l_rank):
                    if rank_src == name_rank:
                        d_preference.loc[i_duty, i_member] = i_rank
    return d_preference

# index: duty, col: member, value: rank of preference
d_preference_main = convert_preference(d_preference_src, l_member_main, l_duty, l_rank)
d_preference_sub = convert_preference(d_preference_src, l_member_sub, l_duty, l_rank)

i_duty_delete = 1
for col in d_preference_main.columns:
    l_temp = d_preference_main[col].tolist()
    if l_temp[i_duty_delete] is not None:
        rank_temp = l_temp[i_duty_delete]
        #l_temp.pop(i_duty_delete)
        l_temp = [i if i < rank_temp else i - 1 for i in l_temp]
        l_temp[i_duty_delete] = None
        d_preference_main[col] = l_temp


# Convert rank quadratically
def quadratic_conversion(d_preference):
    d_preference_conv = d_preference.copy()
    for i in reversed(range(len(d_preference))):
        rank_pre = i
        rank_post = i * (i + 1)/2
        d_preference_conv = d_preference_conv.replace(rank_pre, rank_post)
    return d_preference_conv

d_preference_main = quadratic_conversion(d_preference_main)
d_preference_main = d_preference_main.fillna(0.0)
d_preference_sub = quadratic_conversion(d_preference_sub)

In [29]:
l_member = l_member_sub
d_preference = d_preference_sub
# Initialize model to be optimized
prob_assign = LpProblem()

# Binary assignment variables to be optimized, index: duty, column: member
dv_assign = pd.DataFrame(np.array(addbinvars(len(l_duty), len(l_member))))

# One assignment per member
for i_member in range(len(l_member)):
    prob_assign += (lpSum(dv_assign.loc[:, i_member]) == 1)

# One assignment in specific duty
i_duty_necessary = 1
prob_assign += (lpSum(dv_assign.loc[i_duty_necessary, :]) == 1)

# zero or one assignment in other duties
for i_duty in range(len(l_duty)):
    if i_duty != i_duty_necessary:
        prob_assign += (lpSum(dv_assign.loc[i_duty, :]) >= 0)
        prob_assign += (lpSum(dv_assign.loc[i_duty, :]) <= 1)

# Limit to rank
'''
limit_rank = 2
for i_duty in range(len(l_duty)):
    for i_member in range(len(l_member)):
        prob_assign += (lpDot(dv_assign.loc[i_duty, i_member], d_preference.loc[i_duty, i_member]) <= limit_rank)
'''
# Variable to be minimized (sum of rank)
v_sum_rank = lpSum(lpDot(dv_assign.to_numpy(), d_preference.to_numpy()))
prob_assign += v_sum_rank

# Solve
prob_assign.solve()
v_objective = value(prob_assign.objective)
print('Solved: ' + str(LpStatus[prob_assign.status]) + ', ' + str(v_objective))

# Convert to normal dataframe
d_assign = pd.DataFrame(np.vectorize(value)(dv_assign)).astype(bool)

d_match = pd.DataFrame({'duty': l_duty, 'member': None, 'rank': None})

for i_member, name_member in enumerate(l_member):
    i_duty = d_assign.loc[d_assign[i_member] == True, :].index.tolist()[0]
    rank = d_preference.loc[i_duty, i_member]
    d_match['member'].iloc[i_duty] = name_member
    d_match['rank'].iloc[i_duty] = rank

d_match

Solved: Optimal, 4.0


,duty,member,rank
0,5/2(金)当直,相澤 里佳,0.0
1,5/3(土)日直,押見 陽友,0.0
2,5/3(土)当直,佐藤 未悠,0.0
3,5/4(日)日直,田尻 智哉,3.0
4,5/4(日)当直,None,None
5,5/5(月)日直,None,None
6,5/5(月)当直,None,None
7,5/6(火)日直,南學 正仁,1.0
8,5/6(火)当直,None,None


In [30]:
l_member = l_member_main
d_preference = d_preference_main
# Initialize model to be optimized
prob_assign = LpProblem()

# Binary assignment variables to be optimized, index: duty, column: member
dv_assign = pd.DataFrame(np.array(addbinvars(len(l_duty), len(l_member))))

# One assignment per member
for i_member in range(len(l_member)):
    prob_assign += (lpSum(dv_assign.loc[:, i_member]) == 1)

# One assignment per duty
# zero or one assignment in other duties
i_duty_skip = 1
for i_duty in range(len(l_duty)):
    if i_duty == i_duty_skip:
        prob_assign += (lpSum(dv_assign.loc[i_duty, :]) == 0)
    else:
        prob_assign += (lpSum(dv_assign.loc[i_duty, :]) == 1)


# Limit to rank
'''
limit_rank = 3
for i_duty in range(len(l_duty)):
    for i_member in range(len(l_member)):
        prob_assign += (lpDot(dv_assign.loc[i_duty, i_member], d_preference.loc[i_duty, i_member]) <= limit_rank)
'''
# Variable to be minimized (sum of rank)
v_sum_rank = lpSum(lpDot(dv_assign.to_numpy(), d_preference.to_numpy()))
prob_assign += v_sum_rank

# Solve
prob_assign.solve()
v_objective = value(prob_assign.objective)
print('Solved: ' + str(LpStatus[prob_assign.status]) + ', ' + str(v_objective))

# Convert to normal dataframe
d_assign = pd.DataFrame(np.vectorize(value)(dv_assign)).astype(bool)

d_match = pd.DataFrame({'duty': l_duty, 'member': None, 'rank': None})

for i_member, name_member in enumerate(l_member):
    i_duty = d_assign.loc[d_assign[i_member] == True, :].index.tolist()[0]
    rank = d_preference.loc[i_duty, i_member]
    d_match['member'].iloc[i_duty] = name_member
    d_match['rank'].iloc[i_duty] = rank

d_match

Solved: Optimal, 7.0


,duty,member,rank
0,5/2(金)当直,熊倉 陽介,0.0
1,5/3(土)日直,None,None
2,5/3(土)当直,森田 進,0.0
3,5/4(日)日直,田中 李樹,3.0
4,5/4(日)当直,越山 太輔,1.0
5,5/5(月)日直,池亀 天平,1.0
6,5/5(月)当直,藤川 慎也,1.0
7,5/6(火)日直,清田 正紘,1.0
8,5/6(火)当直,水谷 真志,0.0
